In [ ]:
# doc2vec

import pandas as pd
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# Excel 파일 불러오기
file_path = '/content/drive/MyDrive/분석/ingreDictionary_중복제거및수정.xlsx'  # 파일 경로 수정
df = pd.read_excel(file_path)

# 데이터 전처리 (예: 'description' 열 선택 및 토큰화)
# 실제 열 이름은 파일에 따라 다를 수 있음
processed_docs = [word_tokenize(doc.lower()) for doc in df['Ingredient'] if isinstance(doc, str)]

# TaggedDocument 생성
tagged_data = [TaggedDocument(words=d, tags=[str(i)]) for i, d in enumerate(processed_docs)]

# Doc2Vec 모델 학습
model = Doc2Vec(vector_size=50, window=5, min_count=1, workers=4, epochs=40, dm=0)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

# 모델 저장
model.save("ingre_doc2vec.model")


In [ ]:
# 새로운 재료 텍스트
new_ingredient = "소시지"

# 새로운 재료 텍스트를 토큰화하고 벡터화
new_ingredient_tokens = word_tokenize(new_ingredient.lower())
new_ingredient_vector = model.infer_vector(new_ingredient_tokens)

# 가장 유사한 재료 찾기
similar_ingredients = model.dv.most_similar([new_ingredient_vector], topn=5)

# 결과 출력
for ingredient in similar_ingredients:
    print(f"Ingredient: {df['Ingredient'][int(ingredient[0])]}, Similarity: {ingredient[1]}")


In [ ]:
# word2vec

import pandas as pd
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
nltk.download('punkt')

# Excel 파일 불러오기
file_path = '/content/drive/MyDrive/분석/ingreDictionary_중복제거및수정.xlsx'  # 파일 경로 수정
df = pd.read_excel(file_path)

# 데이터 전처리 (예: 'description' 열 선택 및 토큰화)
# 실제 열 이름은 파일에 따라 다를 수 있음
processed_docs = [word_tokenize(doc.lower()) for doc in df['Ingredient'] if isinstance(doc, str)]

# Word2Vec 모델 학습
model = Word2Vec(sentences=processed_docs, vector_size=50, window=1, min_count=1, workers=4, epochs=40)

# 모델 저장
model.save("ingre_word2vec.model")

# 새로운 재료 텍스트
new_ingredient = "포도"

# 새로운 재료 텍스트를 토큰화하고 벡터화
new_ingredient_tokens = word_tokenize(new_ingredient.lower())
new_ingredient_vector = model.wv[new_ingredient_tokens].flatten()

# 가장 유사한 재료 찾기
similar_ingredients = model.wv.most_similar([new_ingredient_vector], topn=5)

# 결과 출력
for ingredient in similar_ingredients:
    print(f"Ingredient: {ingredient[0]}, Similarity: {ingredient[1]}")


In [ ]:
# TF-IDF

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

# Excel 파일 불러오기
file_path = '/content/drive/MyDrive/분석/ingreDictionary_중복제거및수정.xlsx'
df = pd.read_excel(file_path)

# 데이터 전처리 및 TF-IDF 벡터화
def preprocess_and_vectorize(data):
    processed_docs = [" ".join(word_tokenize(doc.lower())) for doc in data['Ingredient'] if isinstance(doc, str)]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(processed_docs)
    return vectorizer, tfidf_matrix

# 사용자가 입력한 새로운 재료와 가장 유사한 재료 추천
def recommend_similar_ingredient(user_input, vectorizer, tfidf_matrix, data):
    user_input = " ".join(word_tokenize(user_input.lower()))
    user_tfidf_vector = vectorizer.transform([user_input])

    # 새로운 재료와 기존 재료 간의 코사인 유사도 계산
    similarity_scores = cosine_similarity(user_tfidf_vector, tfidf_matrix)

    # 가장 유사한 재료 추출
    most_similar_index = similarity_scores.argmax()
    recommended_ingredient = data.loc[most_similar_index, 'Ingredient']
    similarity_score = similarity_scores[0, most_similar_index]

    return recommended_ingredient, similarity_score

# TF-IDF 벡터화
vectorizer, tfidf_matrix = preprocess_and_vectorize(df)

# 새로운 재료 입력 및 추천
new_ingredient_input = "맛있는간장"  # 사용자가 원하는 새로운 재료를 입력하세요
recommended_ingredient, similarity_score = recommend_similar_ingredient(new_ingredient_input, vectorizer, tfidf_matrix, df)

# 결과 출력
print(f"사용자가 입력한 재료: {new_ingredient_input}")
print(f"추천된 가장 유사한 재료: {recommended_ingredient}")
print(f"유사도 점수: {similarity_score}")


In [ ]:
pip install python-Levenshtein # 편집 거리(Levenshtein Distance)

In [ ]:
# 편집 거리(Levenshtein Distance)

import pandas as pd
import Levenshtein

# Excel 파일 불러오기
file_path = '/content/drive/MyDrive/분석/ingreDictionary_중복제거및수정.xlsx'
df = pd.read_excel(file_path)

# 사용자가 입력한 부분 문자열과 가장 유사한 상위 5개 부분 문자열을 찾는 함수
def find_top5_similar_substrings(user_input, candidates):
    distances = [(Levenshtein.distance(user_input, candidate), candidate) for candidate in candidates]
    distances.sort()
    top5_similar_substrings = distances[:5]

    return top5_similar_substrings

# 예시 데이터로 "Ingredient" 열의 데이터 활용
ingredients = df['Ingredient'].dropna().tolist()

# 사용자가 입력한 부분 문자열
user_input = "다진마늘"

# 부분 문자열 비교를 통해 가장 유사한 상위 5개 부분 문자열 찾기
top5_similar_substrings = find_top5_similar_substrings(user_input, ingredients)

# 결과 출력
print(f"사용자가 입력한 부분 문자열: {user_input}")
print("가장 유사한 상위 5개 부분 문자열:")
for rank, (distance, substring) in enumerate(top5_similar_substrings, start=1):
    print(f"{rank}. {substring}, 편집 거리: {distance}")


In [ ]:
pip install pyspellchecker

In [ ]:
pip install jamo

In [ ]:
from konlpy.tag import Okt
from jamo import h2j, j2hcj

def correct_korean_spelling(input_text):
    okt = Okt()

    # 문장을 형태소 단위로 분리
    morphemes = okt.morphs(input_text)

    # 한글 자모음으로 변환
    jamo_morphemes = [j2hcj(h2j(morpheme)) for morpheme in morphemes]

    # 교정된 형태소들을 다시 문장으로 결합
    corrected_text = ''.join(jamo_morphemes)

    return corrected_text

# 사용자 입력 문자열
user_input = "안녕"

# 한국어 교정 수행
corrected_input = correct_korean_spelling(user_input)

# 결과 출력
print(f"Original input: {user_input}")
print(f"Corrected input: {corrected_input}")


In [ ]:
pip install fuzzywuzzy

In [ ]:
# fuzzywuzzy

import pandas as pd
from fuzzywuzzy import process

# 엑셀 파일 경로
file_path = '/content/drive/MyDrive/분석/ingreDictionary_중복제거및수정.xlsx'

# 재료 사전을 엑셀 파일에서 읽어오기
ingredients_df = pd.read_excel(file_path)
ingredients = ingredients_df['ingre'].tolist()

# 입력된 재료
input_ingredient = '간장'

# 가장 유사한 재료 상위 5개 찾기
matches = process.extract(input_ingredient, ingredients, limit=100)

print(f"입력된 재료: {input_ingredient}")
print("가장 유사한 재료 Top 5:")
for match, score in matches:
    print(f"{match} (유사도: {score}%)")

In [ ]:
import pandas as pd
from fuzzywuzzy import process, fuzz

# 엑셀 파일 경로
file_path = '/content/drive/MyDrive/분석/ingreDictionary_중복제거및수정.xlsx'

# 재료 사전을 엑셀 파일에서 읽어오기
ingredients_df = pd.read_excel(file_path)
ingredients = ingredients_df['Ingredient'].tolist()

# 입력된 재료 파일 경로
input_file_path = '/content/drive/MyDrive/분석/만개의레시피에만있는재료.txt'

# 결과를 저장할 파일 경로
output_file_path = '/content/drive/MyDrive/분석/output_results.txt'

# 입력된 재료 파일 읽어오기 (cp949 인코딩 사용)
with open(input_file_path, 'r', encoding='cp949') as file:
    input_ingredients = file.read().splitlines()

# 결과를 저장할 리스트
results = []

# 각 입력된 재료에 대해 가장 유사한 재료 찾기
for input_ingredient in input_ingredients:
    # Find the most similar result based on both similarity score and similar character length
    similar_results = process.extract(
        input_ingredient,
        ingredients,
        limit=1,
        scorer=lambda x, y: -abs(len(x) - len(y)) + fuzz.WRatio(x, y)
    )

    # 결과를 콘솔에 출력
    result_str = f"입력된 재료: {input_ingredient}\n가장 유사한 재료: {similar_results[0][0]} (유사도: {similar_results[0][1]}%)\n"
    print(result_str)

    # 결과를 리스트에 추가
    results.append(result_str)

# 결과를 파일에 저장
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    output_file.writelines(results)

# 결과 파일 경로 출력
print(f"결과가 다음 경로에 저장되었습니다: {output_file_path}")


In [ ]:
import pandas as pd
from fuzzywuzzy import process, fuzz

# 엑셀 파일 경로
file_path = '/content/drive/MyDrive/분석/식재료_무지성_수정.xlsx'

# 재료 사전을 엑셀 파일에서 읽어오기
ingredients_df = pd.read_excel(file_path)

# 입력된 재료 파일 경로
input_file_path = '/content/drive/MyDrive/분석/만개의레시피에만있는재료.txt'

# 입력된 재료 파일 읽어오기
with open(input_file_path, 'r', encoding='cp949') as file:
    input_ingredients = file.read().splitlines()

# 재료 사전의 각 열을 차례대로 비교
for col in ['식품명', '식품명 세부1', '식품명 세부2', '식품명 세부3', '식품명 세부4', '식품명 세부5', '식품명 세부6', '식품명 세부7', '식품대분류', '식품상세분류']:
    found_match = False
    selected_ingredients = []

    for input_ingredient in input_ingredients:
        matches = process.extract(input_ingredient, ingredients_df[col], limit=1, scorer=fuzz.ratio)
        best_match = matches[0]

        # 일정 유사도 이상인 경우에만 선택
        if best_match[1] >= 70:
            selected_ingredients.append(best_match[0])
            found_match = True

    if found_match:
        print(f"입력된 재료: {input_ingredient}")
        print(f"가장 유사한 {col}: {selected_ingredients}\n")
    else:
        print(f"입력된 재료에 대한 일치하는 {col}을 찾을 수 없습니다.\n")


In [ ]:
import pandas as pd
from fuzzywuzzy import process, fuzz

# 엑셀 파일 경로
file_path = '/content/drive/MyDrive/분석/식재료_무지성_수정.xlsx'

# 재료 사전을 엑셀 파일에서 읽어오기
ingredients_df = pd.read_excel(file_path)

# 입력된 재료 파일 경로
input_file_path = '/content/drive/MyDrive/분석/만개의레시피에만있는재료.txt'

# 입력된 재료 파일 읽어오기 (utf-8 인코딩 사용)
with open(input_file_path, 'r', encoding='cp949') as file:
    input_ingredients = file.read().splitlines()

# 입력된 재료 확인
print("입력된 재료:")
print(input_ingredients)

# 재료 사전의 각 열을 차례대로 비교
for col in ['식품명', '식품명 세부1', '식품명 세부2', '식품명 세부3', '식품명 세부4', '식품명 세부5', '식품명 세부6', '식품명 세부7', '식품대분류', '식품상세분류']:
    found_match = False
    selected_ingredients = []

    for input_ingredient in input_ingredients:
        matches = process.extract(input_ingredient, ingredients_df[col], limit=1, scorer=fuzz.ratio)
        best_match = matches[0]

        # 일정 유사도 이상인 경우에만 선택
        if best_match[1] >= 70:
            selected_ingredients.append(best_match[0])
            found_match = True

    if found_match:
        print(f"입력된 재료: {input_ingredient}")
        print(f"가장 유사한 {col}: {selected_ingredients}\n")
    else:
        print(f"입력된 재료에 대한 일치하는 {col}을 찾을 수 없습니다.\n")
